In [172]:
from trdg.generators import (
    GeneratorFromDict,
    GeneratorFromRandom,
    GeneratorFromStrings,
    GeneratorFromWikipedia,
)
from random import sample
from itertools import combinations, permutations


folder = '/Users/liuliangjun/Downloads/images/vaild'

ss = '0123456789 '
_len_8 = [i for i in map(lambda x: ''.join(x), permutations(ss, 8))]
_len_7 = [i for i in map(lambda x: ''.join(x), permutations(ss, 7))]
_len_7 = [i for i in map(lambda x: ''.join(x), permutations(ss, 7))]
_len_6 = [i for i in map(lambda x: ''.join(x), permutations(ss, 6))]
_len_5 = [i for i in map(lambda x: ''.join(x), permutations(ss, 5))]
_len_4 = [i for i in map(lambda x: ''.join(x), permutations(ss, 4))]
_len_3 = [i for i in map(lambda x: ''.join(x), permutations(ss, 3))]
_len_2 = [i for i in map(lambda x: ''.join(x), permutations(ss, 2))]


lst = [*sample(_len_6, 200), *sample(_len_7, 200), *sample(_len_8, 200)]
# lst = [*_len_2, *sample(_len_3, 300), *sample(_len_4, 500), *sample(_len_6, 800), *sample(_len_7, 1000)]
# lst = sample(_len_5, 20)
lst = [i.strip() for i in lst]
print(len(lst))


generator = GeneratorFromStrings(lst, width=150, count=len(lst))

count = 0

for img, lbl in generator:
    img.save(os.path.join(folder, f'{count}_{lbl}.png'))
    count += 1

count

600


600

In [173]:
# alphabet = u'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ~!@#$%^&*?-_+=()[]{}/|\<>,.;:\'" '
alphabet = '0123456789 '
len(alphabet)

11

In [175]:
from imutils import paths
import cv2
import random
from keras.preprocessing.image import img_to_array
import numpy as np
import matplotlib

def text_to_labels(text,alphabet):
    ret = []
    for char in text:
        ret.append(alphabet.find(char))
    return ret

def loadpic(path,batchSize,absolute_max_string_len,downsample_factor):
    imgPaths=list(paths.list_images(path))
    random.shuffle(imgPaths) 
    print(len(imgPaths))
    random.seed(20)

    while 1:
        x_batch=[]#图片
        flag = False
        y_batch=np.ones([batchSize, absolute_max_string_len]) * -1 # 图片标签变成编码
        input_length = np.zeros([batchSize, 1]) # 图片下采样后进入GRN的尺寸
        label_length=np.zeros([batchSize, 1]) # 图片中字符的长度
        labels_batch=[] # 图片的真是标签 字符串
        for i in range(batchSize):

            imgpath=imgPaths[random.randint(0,len(imgPaths)-1)]
            # print(imgpath)
            try:
                img=cv2.imread(imgpath,0)
                img=cv2.resize(img,(150,32))
            except:
                print(imgpath)
                flag = True
                break
            
            # cv2.imshow("ig",img)
            # cv2.waitKey(30)
            img=img_to_array(img)
            x_batch.append(img)
            
            input_length[i]=np.array(img).shape[1]//downsample_factor

            # imgname=imgpath[imgpath.rindex('\\')+1:imgpath.rindex('.')]
            imgname=imgpath.split('_')[-1].split('.')[0]
            labels_batch.append(imgname)

            label_length[i]=len(imgname)

            y_batch[i,0:len(imgname)]=text_to_labels(imgname, alphabet)

        if flag:
            flag = False
            continue

        x_batch = np.array(x_batch, dtype="float") / 255.0 - 0.5
#         x_batch= x_batch.swapaxes(1,2)

        inputs = {'the_input': x_batch,
                  'the_labels': y_batch,
                  'input_length': input_length,
                  'label_length': label_length,
                  'source_str': labels_batch  # used for visualization only # 可视化使用
                  }
        outputs = {'ctc': np.zeros([batchSize])}  
        yield (inputs,outputs)
        
# https://github.com/felixBrave/ocr_chinese/blob/master/crnn/net/training.py

In [39]:
from keras import backend as K
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Input, Dense, Activation, BatchNormalization, Dropout
from keras.layers import Reshape, Lambda
from keras.layers.merge import add, concatenate
from keras.models import Model
from keras.layers.recurrent import GRU


class OCRNet:
    @staticmethod
    def ctc_lambda_func(args):
        y_pred, labels, input_length, label_length = args
        # 注意这里若使用y_pred会导致sequence_length（0）<= X.
        # 注意理解sequence_length(0)这个东西  这里是32  因为第41句中的img_w//poolsize（2）是sequence_length（0）
        y_pred = y_pred[:, :, :]
        # 这里labels是实际的标签，y_pred是预测出来的标签，input_length是预测标签的长度，label_length是实际标签的长度
        return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

    @staticmethod
    def build(img_w, img_h, depth, alphabetLength, absolute_max_string_len):
        conv_filters = 32  
        kernel_size = (3, 3)
        pool_size = (2, 2)
        time_dense_size = 128
        rnn_size = 128  

        act = 'relu'
        if K.image_data_format() == 'channels_first':
            input_shape = (depth, img_w, img_h)
        else:
            input_shape = (img_w, img_h, depth)

        input_data = Input(name='the_input', shape=input_shape, dtype='float32')
        
        # 参考VGG19网络，双层卷积层再接一层池化层
        
        inner = Conv2D(conv_filters, kernel_size, padding='same', activation=act, name='conv1')(input_data)
        inner = Conv2D(conv_filters, kernel_size, padding='same', activation=act,  name='conv2')(inner)
        inner = BatchNormalization()(inner)
        inner = MaxPooling2D(pool_size=pool_size, name='max1')(inner)

        inner = Conv2D(conv_filters*2, kernel_size, padding='same', activation=act, name='conv3')(inner)
        inner = Conv2D(conv_filters*2, kernel_size, padding='same', activation=act, name='conv4')(inner)
        inner = BatchNormalization()(inner)
        inner = MaxPooling2D(pool_size=pool_size, name='max2')(inner)

        inner = Conv2D(conv_filters*4, kernel_size, padding='same', activation=act, name='conv5')(inner)
        inner = Conv2D(conv_filters*4, kernel_size, padding='same', activation=act, name='conv6')(inner)
        inner = BatchNormalization()(inner)
        inner = MaxPooling2D(pool_size=pool_size, name='max3')(inner)
        
        inner = Conv2D(conv_filters*4, kernel_size, padding='same', activation=act, name='conv7')(inner)
        inner = Conv2D(conv_filters*4, kernel_size, padding='same', activation=act, name='conv8')(inner)
        inner = BatchNormalization()(inner)
        inner = MaxPooling2D(pool_size=pool_size, name='max4')(inner)
        
        conv_to_rnn_dims = (img_w // (pool_size[0] ** 4), (img_h // (pool_size[0] ** 4)) * conv_filters*4)
        inner = Reshape(target_shape=conv_to_rnn_dims, name='reshape')(inner)
        inner = Dense(time_dense_size, activation=act, name='dense1')(inner)

        gru_1 = GRU(rnn_size, return_sequences=True,
                    kernel_initializer='he_normal', name='gru1')(inner)
        gru_1b = GRU(rnn_size, return_sequences=True,
                     go_backwards=True, kernel_initializer='he_normal',
                     name='gru1_b')(inner)
        gru1_merged = add([gru_1, gru_1b])

        gru_2 = GRU(rnn_size, return_sequences=True,
                    kernel_initializer='he_normal', name='gru2')(gru1_merged)
        gru_2b = GRU(rnn_size, return_sequences=True, go_backwards=True,
                     kernel_initializer='he_normal', name='gru2_b')(gru1_merged)
        
        # 多添加一层Dense
        inner = Dropout(0.5)(concatenate([gru_2, gru_2b]))
        inner = Dense(256, name='dense2')(inner)

        # transforms RNN output to character activations:
        # 是len(alphabet)+1 还是 len(alphabet) 为何？？？
        inner = Dense(len(alphabet), name='dense3')(inner)
        y_pred = Activation('softmax', name='softmax')(inner)

        base_model = Model(inputs=input_data, outputs=y_pred)
        # base_model.summary()

        labels = Input(name='the_labels', shape=[absolute_max_string_len], dtype='float32')
        input_length = Input(name='input_length', shape=[1], dtype='int64')
        label_length = Input(name='label_length', shape=[1], dtype='int64')

        loss_out = Lambda(OCRNet.ctc_lambda_func, output_shape=(1,), name='ctc')(
            [y_pred, labels, input_length, label_length])

        train_model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)
        train_model.summary()
        return base_model, train_model

In [40]:
absolute_max_string_len = 36
base_model, train_model = OCRNet.build(280, 32, 1, len(alphabet), absolute_max_string_len)

Model: "model_27"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 280, 32, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 280, 32, 32)  320         the_input[0][0]                  
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 280, 32, 32)  9248        conv1[0][0]                      
__________________________________________________________________________________________________
batch_normalization_12 (BatchNo (None, 280, 32, 32)  128         conv2[0][0]                      
___________________________________________________________________________________________

In [176]:
# 是

import os,sys
# parentdir = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
# sys.path.append(parentdir)

from PIL import Image

import keras.backend as K
import numpy as np

from keras.layers import Flatten, BatchNormalization, Permute, TimeDistributed, Dense, Bidirectional, GRU, LSTM
from keras.layers import Input, Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Lambda
from keras.models import Model
from keras.optimizers import SGD, Adam, Adadelta, RMSprop

img_h = 32

# n_class 如果不加1会报错  Saw a non-null label (index >= num_classes - 1) following a null label,
n_class = len(alphabet)+1

def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

# def ctc_lambda_func(args):
#     y_pred, labels, input_length, label_length = args
#     # 注意这里若使用y_pred会导致sequence_length（0）<= X.
#     # 注意理解sequence_length(0)这个东西  这里是32  因为第41句中的img_w//poolsize（2）是sequence_length（0）
#     y_pred = y_pred[:, :, :]
#     # 这里labels是实际的标签，y_pred是预测出来的标签，input_length是预测标签的长度，label_length是实际标签的长度
#     return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

def crnn_network(height=img_h, nclass=n_class):
    '''
    cnn + rnn + ctc model !!!
    :param height:
    :param nclass:
    :return:
    '''
    input = Input(shape=(height, None, 1), name='the_input')
    # CNN
    m = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same', name='conv1')(input)
    m = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool1')(m)
    m = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same', name='conv2')(m)
    m = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool2')(m)
    m = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', name='conv3')(m)
    m = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', name='conv4')(m)

    m = ZeroPadding2D(padding=(0, 1))(m)
    m = MaxPooling2D(pool_size=(2, 2), strides=(2, 1), padding='valid', name='pool3')(m)

    m = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', name='conv5')(m)
    m = BatchNormalization(axis=3)(m)
    m = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', name='conv6')(m)
    m = BatchNormalization(axis=3)(m)
#     m = ZeroPadding2D(padding=(0, 1))(m)
    m = MaxPooling2D(pool_size=(2, 2), strides=(2, 1), padding='valid', name='pool4')(m)
#     m = Conv2D(512, kernel_size=(2, 2), activation='relu', padding='valid', name='conv7')(m)

    # m的输出维度为(h, w, c) -> (1, w/4, 512) 转换 (w, b, c) = (seq_len, batch, input_size)
    m = Permute((2, 1, 3), name='permute')(m)
    m = TimeDistributed(Flatten(), name='timedistrib')(m)

    # RNN 
    # LSTM GRU 为何参数不一样？
    m = Bidirectional(GRU(64, return_sequences=True), name='blstm1')(m)
    m = Dense(64, name='blstm1_out', activation='linear')(m)
    m = Bidirectional(GRU(64, return_sequences=True), name='blstm2')(m)
    
    y_pred = Dense(nclass, name='blstm2_out', activation='softmax')(m)

    basemodel = Model(inputs=input, outputs=y_pred)
    #basemodel.summary()
    
    # max_len
    max_len = None # None  12
    labels = Input(name='the_labels', shape=[max_len], dtype='float32')
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')
    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])

    model = Model(inputs=[input, labels, input_length, label_length], outputs=[loss_out])

    # sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5)
    # model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=sgd, metrics=['accuracy'])
    # model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer="adadelta", metrics=['accuracy'])
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer='adam', metrics=['accuracy'])
#     model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer='rmsprop', metrics=['accuracy'])

    model.summary()

    return model, basemodel

def predict(img_path, model):
    """
    输入图片，输出keras模型的识别结果
    :param img_path:
    :return:
    """
    img = Image.open(img_path)
    img = img.convert('L')

    scale = img.size[1] * 1.0 / 32
    w = int(img.size[0] / scale)
    img = img.resize((w, 32), Image.BILINEAR)
    img = np.array(img).astype(np.float32) / 255.0 - 0.5
    X = img.reshape((32, w, 1))
    X = np.array([X])

    y_pred = model.predict(X)

    y_pred = y_pred[:, :, :]

    out = K.get_value(K.ctc_decode(y_pred, input_length=np.ones(y_pred.shape[0]) * y_pred.shape[1], )[0][0])[:, :]
    out_s = u''.join([char[x] for x in out[0]])

    return out_s


In [177]:
model, basemodel = crnn_network()

Model: "model_65"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 32, None, 1)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, None, 16) 160         the_input[0][0]                  
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 16, None, 16) 0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 16, None, 16) 2320        pool1[0][0]                      
___________________________________________________________________________________________

In [191]:
from keras.callbacks import ModelCheckpoint
save_path = 'base_model.h5'

if os.path.exists(save_path):
    try:
        model.load_weights(save_path)
    except Exception as e:
        print('Load weights raise exception.')
        print(e)

checkpoint = ModelCheckpoint(save_path, 
                             monitor='val_loss',
                             verbose=1, 
                             save_weights_only=True)
base_path = 'images/'
train_folder = '/Users/liuliangjun/Downloads/images/train/'
vaild_folder = '/Users/liuliangjun/Downloads/images/vaild/'
max_len = 16
model.fit_generator(generator=loadpic(train_folder, 16, 16, 2**4),
                    steps_per_epoch=500, 
                    epochs=5,
                    callbacks=[checkpoint],
                    validation_data=loadpic(vaild_folder, 16, 16, 2**4),
                    validation_steps=50,
                    )


basemodel.save('predict_model.h5')
    

Epoch 1/5
600
8130
500/500 [==============================] - 66s 132ms/step - loss: 0.4296 - accuracy: 0.8765 - val_loss: 1.9906 - val_accuracy: 0.4325

Epoch 00001: saving model to base_model.h5
Epoch 2/5
500/500 [==============================] - 66s 132ms/step - loss: 0.3903 - accuracy: 0.8856 - val_loss: 3.1909 - val_accuracy: 0.4863

Epoch 00002: saving model to base_model.h5
Epoch 3/5
500/500 [==============================] - 67s 133ms/step - loss: 0.3644 - accuracy: 0.9018 - val_loss: 0.6526 - val_accuracy: 0.5475

Epoch 00003: saving model to base_model.h5
Epoch 4/5
500/500 [==============================] - 67s 133ms/step - loss: 0.3520 - accuracy: 0.9078 - val_loss: 2.6449 - val_accuracy: 0.5150

Epoch 00004: saving model to base_model.h5
Epoch 5/5
500/500 [==============================] - 67s 133ms/step - loss: 0.3024 - accuracy: 0.9201 - val_loss: 2.6559 - val_accuracy: 0.4613

Epoch 00005: saving model to base_model.h5


In [192]:
from keras.models import load_model
my_model=load_model('predict_model.h5', custom_objects = {'<lambda>': lambda y_true, y_pred: y_pred})


In [193]:
def predict(img_path, model):
    """
    输入图片，输出keras模型的识别结果
    :param img_path:
    :return:
    """
    img = Image.open(img_path)
    img = img.convert('L')

    scale = img.size[1] * 1.0 / 32
    w = int(img.size[0] / scale)
    img = img.resize((w, 32), Image.BILINEAR)
    img = np.array(img).astype(np.float32) / 255.0 - 0.5
    X = img.reshape((32, w, 1))
#     print(X.shape)
    X = np.array([X])
    
#     print(X.shape)

    y_pred = model.predict(X)

    y_pred = y_pred[:, :, :]

    out = K.get_value(K.ctc_decode(y_pred, input_length=np.ones(y_pred.shape[0]) * y_pred.shape[1], )[0][0])[:, :]
    out_s = u''.join([char[x] for x in out[0]])

    return out_s

In [194]:
predict('/Users/liuliangjun/Downloads/images/3_403 9.png', my_model)

NameError: name 'char' is not defined

In [196]:
from keras.models import Model
import cv2
from keras.preprocessing.image import img_to_array
import numpy as np
from keras.models import load_model
from keras import backend as K


def labels_to_text(labels):
    ret = []
    for c in labels:
        if c == len(alphabet):  # CTC Blank
            ret.append("")
        else:
            ret.append(alphabet[c])
    return "".join(ret)

def decode_predict_ctc(out, top_paths = 1):
    results = []
    beam_width = 5
    if beam_width < top_paths:
      beam_width = top_paths
    for i in range(top_paths):
      lables = K.get_value(K.ctc_decode(out, input_length=np.ones(out.shape[0])*out.shape[1],
                           greedy=False, beam_width=beam_width, top_paths=top_paths)[0][i])[0]
      text = labels_to_text(lables)
      results.append(text)
    return results

In [199]:
# predict('/Users/liuliangjun/Downloads/images/3_403 9.png', my_model)

img=cv2.imread('/Users/liuliangjun/Downloads/images/1_61978.png', 0)
img=cv2.resize(img,(124,32))
img=img_to_array(img)
img=np.array(img,dtype='float')/255.0 - 0.5
img=np.expand_dims(img, axis=0)
# img=img.swapaxes(1,2)   
print(img.shape)
top_pred_texts = decode_predict_ctc(my_model.predict(img))
top_pred_texts

(1, 32, 124, 1)


['61978']

In [204]:
nums = 10
name = 'name'
print(f'nums: {nums}, name: {name}')
'afa {}, {}'.format(10-9, 90)

nums: 10, name: name


'afa 1, 90'